In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv('/home/seungah/crwlnt/CB/inter_240113_title_none_piv.csv')
df = df[df['local_time'] >= '2024-01-15 00:00:00+00:00']
df

In [ ]:
with open('/home/seungah/crwlnt/CB/product_info_with_title.pickle', 'rb') as fr:
    product_data = pickle.load(fr)

product_data

## hashed_ip 별 title 묶기

In [ ]:
data = df.copy()
data_drop_local_time = data.drop(columns='local_time', axis=0)
grouped = data_drop_local_time.groupby('hashed_ip')

In [ ]:
df_grouped = pd.DataFrame(columns=['hashed_ip', 'products', 'product_titles', 'last_product', 'last_product_title'])

for i in tqdm(grouped):
    if len(i[1]) > 1 :
        df_grouped.loc[len(df_grouped)] = [i[0], list(set(i[1]['products'][:-1])), str(list(set(i[1]['title'][:-1]))), list(set(i[1]['products'][-1:])), str(list(set(i[1]['title'][-1:])))]

In [ ]:
df_grouped['product_titles'] = df_grouped['product_titles'].map(lambda x: x.replace('[','').replace(']','').replace("'",'').replace(',','').replace('(', ' ').replace(')', ' '))
df_grouped['product_titles'] = df_grouped['product_titles'].map(lambda x: x.split(' '))
df_grouped['product_titles'] = df_grouped['product_titles'].map(lambda x: ' '.join(x).split())
df_grouped['product_titles'] = df_grouped['product_titles'].map(lambda x: ' '.join(x))

df_grouped['last_product_title'] = df_grouped['last_product_title'].map(lambda x: x.replace('[','').replace(']','').replace("'",'').replace(',','').replace('(', ' ').replace(')', ' '))
df_grouped['last_product_title'] = df_grouped['last_product_title'].map(lambda x: x.split(' '))
df_grouped['last_product_title'] = df_grouped['last_product_title'].map(lambda x: ' '.join(x).split())
df_grouped['last_product_title'] = df_grouped['last_product_title'].map(lambda x: ' '.join(x))
df_grouped

## TF-IDF

In [ ]:
vectorizer = TfidfVectorizer()

tfidf_user = vectorizer.fit_transform(df_grouped['product_titles'])
tfidf_product = vectorizer.transform(product_data['title'])

dtm_user = np.array(tfidf_user.todense())
dtm_product = np.array(tfidf_product.todense())

cosine_similarities = cosine_similarity(dtm_product, dtm_user)
sim = pd.DataFrame(cosine_similarities) # 유저-아이템 유사도 행렬

In [ ]:
hit100 = 0
df_grouped['top100'] = None
for i in tqdm(range(len(df_grouped))):
    # i 번째 유저와의 유사도 top100 item index
    product_idx = list(sim[i].sort_values(ascending=False).index[:100])

    # index를 통해 가져온 product title + 그냥 해보는 df_grouped에 top100 결과 추가하기
    result = product_data.iloc[product_idx, :]
    df_grouped.loc[i, 'top100'] = str(list(result['title']))

    # 10개에 이 유저가 마지막으로 본 아이템이 속해 있는가? 있으면 hit10에 +1
    test = result[result['title'].str.contains(df_grouped.loc[i, 'last_product_title'])]
    if len(test):
        hit100 +=1

print("hit@100 : ", hit100/len(df_grouped))

## user_idx, item_idx pickle 만들기

In [ ]:
user_idx = {i:df_grouped.loc[i,'hashed_ip'] for i in tqdm(range(len(df_grouped)))}
item_idx = {i:product_data.loc[i,'id'] for i in tqdm(range(len(product_data)))}

In [ ]:
# with open('user_idx.pickle','wb') as fw:
#     pickle.dump(user_idx, fw)

In [ ]:
# with open('item_idx.pickle','wb') as fw:
#     pickle.dump(item_idx, fw)